In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import math

from sklearn.metrics import log_loss, make_scorer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold

from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

import optuna
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,precision_score,recall_score,accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from lightgbm import plot_metric
import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from functools import partial

In [3]:
def balanced_log_loss(y_pred,y_true):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_true = y_true.get_label()
    nc = np.bincount(y_true.astype(int))

    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / 2
    return logloss

def balanced_log_loss2(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    nc = np.bincount(y_true.astype(int))
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(1-y_pred))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred)))) / 2
    return logloss

def xgb_metric(y_pred,y_true):
    return 'balanced_log_loss', balanced_log_loss(y_pred,y_true)

def lgb_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss2(y_true, y_pred), False

In [4]:
import os
import random
def seed_everything(seed=None):
    '''

    固定seed
    :param seed: int, 随机种子
    '''
    max_seed_value = np.iinfo(np.uint32).max
    min_seed_value = np.iinfo(np.uint32).min

    if (seed is None) or not (min_seed_value <= seed <= max_seed_value):
        seed = random.randint(np.iinfo(np.uint32).min, np.iinfo(np.uint32).max)
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.benchmark = False
    # torch.backends.cudnn.deterministic = True
    return seed
seed_everything(42)

42

In [5]:
class FeatureCreator():
    def __init__(self, add_attributes=True):
        
        self.add_attributes = add_attributes
        
    def fit(self, X, y=None):
        
        return self
    
    def transform(self, X):
        if self.add_attributes:
            X_copy = X.copy()
            X_copy = self.outlier(X_copy)
            X_copy = self.base(X_copy)
            X_copy = self.encoder(X_copy)
            return X_copy
        else:
            return X_copy
        
    def base(self, X):
        X['out_GL'] = 0
        X.loc[X['GL']<1,'out_GL'] = X.loc[X['GL']<1,'GL'].map(lambda x : x-X.loc[X['GL']<1,'GL'].mean())
        X.loc[X['GL']>1.5,'out_GL'] = X.loc[X['GL']>1.5,'GL'].map(lambda x : x-X.loc[X['GL']>1.5,'GL'].mean())
        X.out_GL = X.out_GL.astype('float')
        X['DA*CS'] = np.log(X.DA*2 / X.CS**0.5)
        return X
    
    def outlier(self, X):
        return X
        
    def fill_na(self, X):
        # X['EH'].loc[pd.isnull(X['EH'])] = X['EH'][X['EJ']=='B'].mean()
        # print(X['EH'][X['EJ']=='B'].mean())
        return X.fillna(X.median())
    
    def encoder(self, X):
        try:
            X['EJ'] = X['EJ'].map({'A': 0, 'B': 1})
            # X['BQ_NA'] = 0
            # X.loc[pd.isnull(X.BQ),'BQ_NA'] = 1
            return X
        except:
            return X

ft = ['Id','AF','BQ','AB','DU','DI','FL','CR','DH','DA','EH','CD ','BP','BC','DL','EE','FD ','DE','GL','FR','FI','EB','CU','CS']

train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")


train = train[ft+['Class']]
test = test[ft]

FT = FeatureCreator(add_attributes=True)
train = FT.transform(train)
test = FT.transform(test)

drop_ft = ['DI','FD ','CS']

train = train.drop(columns = drop_ft)#
test = test.drop(columns = drop_ft)

df, test_df = train.copy(), test.copy()

scaler = StandardScaler()
col = train.drop(columns = ['Id','Class']).columns.values
train.loc[:,col] = scaler.fit_transform(train.loc[:,col])
test.loc[:,col] = scaler.transform(test.loc[:,col])

def data22(stand = True):
    ft = ['AF','BQ','AB','DU','DI','FL','CR','DH','DA','EH','CD ','BP','BC','DL','EE','FD ','DE','GL','FR','FI','EB','CU','CS']
    train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
    test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
    greeks = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv")
    
    train = train[ft+['Class']]
    test = test[ft]

    FT = FeatureCreator(add_attributes=True)
    train = FT.transform(train)
    test = FT.transform(test)
    
    drop_ft = ['DI','FD ','CS']
    train = train.drop(columns = drop_ft)
    test = test.drop(columns = drop_ft)
    
    if stand:
        scaler = StandardScaler()
        col = train.drop(columns = ['Class']).columns.values
        train.loc[:,col] = scaler.fit_transform(train.loc[:,col])
        test.loc[:,col] = scaler.transform(test.loc[:,col])
    return {'train':train, 'test':test}

In [6]:
model_data = {}
xgb_22_p1 = XGBClassifier(n_estimators = 10000,custom_metric = lgb_metric,n_jobs = -1,booster = 'gbtree',random_state = 42,tree_method = 'hist',objective = 'binary:logistic',
                          **{'lambda': 0.00914022836565986,
                             'alpha': 0.01340407501363288,
                             'colsample_bytree': 0.5712493876996746,
                             'subsample': 0.5449231181998039,
                             'learning_rate': 0.04055159194141396,
                             'max_depth': 6,
                             'min_child_weight': 9,
                             'max_leaves': 63,
                             'max_bin': 50})
model_data['xgb_22_p1'] = {'model':xgb_22_p1,'data':data22(False)}

lgb_22_p1 = LGBMClassifier(boosting_type='goss', random_state=42,class_weight='balanced',verbose=-1,n_estimators = 10000,
                         metric='none', is_unbalance=True,**{'reg_lambda': 0.030365282809260312,
                 'reg_alpha': 0.01799997830170692,
                 'colsample_bytree': 0.5520592747396248,
                 'subsample': 0.36452286036281173,
                 'learning_rate': 0.06412993557201371,
                 'max_depth': 12,
                 'min_child_samples': 6,
                 'num_leaves': 127,
                 'max_bin': 50})

model_data['lgb_22_p1'] = {'model':lgb_22_p1,'data':data22(True)}

lgb_22_p2 =  LGBMClassifier(boosting_type='goss', learning_rate=0.06733232950390658, n_estimators = 50000, 
                          early_stopping_round = 300, random_state=42,
                        subsample=0.6970532011679706,
                        colsample_bytree=0.6055755840633003,
                         class_weight='balanced',
                         metric='none', is_unbalance=True, max_depth=8)

model_data['lgb_22_p2'] = {'model':lgb_22_p2,'data':data22(False)}

In [7]:
%%time
import xgboost as xgb
def cv_xgb(train, test, target,model, kfold = 10):
    weights = []
    test_preds = np.zeros((len(test)))
    cv = 0
    kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    train_result = train.Class.copy()
    # split_key = train['sum']
    train_targets = train[target]
    train = train.drop(columns = [target])
    xgb_loss = 0
    final_result = []
    Ensemble_MAE = 0
    score = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
        print(f"------------> Fold {fold + 1} <-----------------")

        X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
        y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
        
        model.set_params(scale_pos_weight = sum(y_train == 0) / sum(y_train == 1))
       
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=300,verbose=0,eval_metric= xgb_metric)
        xgb_preds = model.predict_proba(X_valid)
        xgb_test_preds = model.predict_proba(test)
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            XGBoost                 |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"BLL: {balanced_log_loss2(y_valid,xgb_preds[:,1])}")
        xgb_loss += balanced_log_loss2(y_valid[:],xgb_preds[:,1])

        meta_train = [xgb_preds]#ctb_preds, , xgb_preds
        meta_test = [xgb_test_preds]#ctb_test_preds, , xgb_test_preds
        
        final_result.append(meta_test)
        train_result[valid_idx] = xgb_preds[:,1]
        # weights.append(2/(lgb_loss+xgb_loss))
        weights.append(1/(balanced_log_loss2(y_valid,xgb_preds[:,1])))
        score.append(balanced_log_loss2(y_valid,xgb_preds[:,1]))
    print(f'feature is {X_valid.columns.values}')
    print(f"Average xgb is : {xgb_loss/kfold}")
    print(f"std is {np.std(score)}")
    print(f"total is {np.std(score) + xgb_loss/kfold}")
    return train_result,final_result,weights


def cv_lgb(train, test, target,model,kfold = 10):
    weights = []
    test_preds = np.zeros((len(test)))
    cv = 0
    kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    train_result = train.Class.copy()
    # split_key = train['sum']
    train_targets = train[target]
    train = train.drop(columns = [target])
    lgb_loss = 0
    final_result = []
    Ensemble_MAE = 0
    score = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
        print(f"------------> Fold {fold + 1} <-----------------")

        X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
        y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
        
        
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=300,verbose=-1,eval_metric= lgb_metric)
        # plot_metric(lgb,'mae')
        lgb_preds = model.predict_proba(X_valid)
        lgb_test_preds = model.predict_proba(test)
        
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            LightGBM                |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"BLL: {balanced_log_loss2(y_valid,lgb_preds[:,1])}")
        lgb_loss += balanced_log_loss2(y_valid[:],lgb_preds[:,1])

        meta_train = [lgb_preds]#ctb_preds, , xgb_preds
        meta_test = [lgb_test_preds]#ctb_test_preds, , xgb_test_preds
        
        final_result.append(meta_test)
        train_result[valid_idx] = lgb_preds[:,1]
        # weights.append(2/(lgb_loss+xgb_loss))
        weights.append(1/(balanced_log_loss2(y_valid,lgb_preds[:,1])))
        score.append(balanced_log_loss2(y_valid,lgb_preds[:,1]))
    print(f'feature is {X_valid.columns.values}')
    print(f"Average lgb is : {lgb_loss/kfold} ")
    print(f"std is {np.std(score)}")
    print(f"total is {np.std(score) + lgb_loss/kfold}")
    return train_result,final_result,weights

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 15.5 µs


In [8]:
def get_test(meta_test,weight,size):
    test_preds = np.zeros((size,2))
    for i in range(10):
        test_preds[:, 0] += weight[i] * meta_test[i][0][:, 0]
        test_preds[:, 1] += weight[i] * meta_test[i][0][:, 1]
    test_preds /= sum(weight)
    return test_preds[:,1]

In [9]:
train_stack = pd.DataFrame({'Class':model_data['lgb_22_p1']['data']['train'].Class})
test_size = model_data['lgb_22_p1']['data']['test'].shape[0]
test_stack = pd.DataFrame({'tmp':np.zeros((test_size))})
for i in model_data.keys():
    if 'xgb' in i:
        tmp_train,tmp_test,weight = cv_xgb(model_data[i]['data']['train'],model_data[i]['data']['test'],'Class',model_data[i]['model'])
        test_stack[i] = get_test(tmp_test,weight,test_size)
        train_stack[i] = tmp_train
    elif 'lgb' in i:
        tmp_train,tmp_test,weight = cv_lgb(model_data[i]['data']['train'],model_data[i]['data']['test'],'Class',model_data[i]['model'])
        test_stack[i] = get_test(tmp_test,weight,test_size)
        train_stack[i] = tmp_train
      

------------> Fold 1 <-----------------
|------------------------------------|
|                                    |
|            XGBoost                 |
|                                    |
|------------------------------------|
 
------------- Train -----------------
BLL: 0.27535879324147156
------------> Fold 2 <-----------------
|------------------------------------|
|                                    |
|            XGBoost                 |
|                                    |
|------------------------------------|
 
------------- Train -----------------
BLL: 0.12800379947887558
------------> Fold 3 <-----------------
|------------------------------------|
|                                    |
|            XGBoost                 |
|                                    |
|------------------------------------|
 
------------- Train -----------------
BLL: 0.2722688386160316
------------> Fold 4 <-----------------
|------------------------------------|
|                     

In [10]:
fold_idx = train.Class
train_targets = train.Class
kf = StratifiedKFold(n_splits=10 ,shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
        fold_idx.iloc[valid_idx] = fold
train_stack['fold'] = fold_idx


In [11]:
import optuna
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,precision_score,recall_score,accuracy_score
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression, Ridge
import lightgbm
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from lightgbm import plot_metric
import random
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
from functools import partial
class OptunaWeights:
    def __init__(self, random_state, n_trials=100):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 1e-15, 1) for n in range(len(y_preds))]
        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)
        #print(weighted_pred)
        # Calculate the score for the weighted prediction
        score = balanced_log_loss2(y_true,weighted_pred)
        return score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='minimize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T,axis = 1,weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights

In [12]:
%%time
from sklearn.linear_model import LogisticRegression

def cv_stacking(train, test, target,model, kfold = 10):
    weights = []
    test_preds = np.zeros((len(test)))
    cv = 0
    kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
    train_result = train.Class.copy()
    # split_key = train['sum']
    train_targets = train[target]
    train = train.drop(columns = [target])
    stacking_loss = 0
    final_result = []
    score = []
    fold_idx = train.fold
    train = train.drop(columns = ['fold'])
    for fold in range(0,kfold):
        meta_train = []
        meta_test = []
        print(f"------------> Fold {fold + 1} <-----------------")
        valid_idx = fold_idx==fold
        train_idx = fold_idx!=fold
        X_train, X_valid = train[train_idx], train[valid_idx]
        y_train, y_valid = train_targets[train_idx], train_targets[valid_idx]
        
        for k in X_valid:
            meta_train.append(X_valid[k].tolist())
            meta_test.append(test[k].tolist())
        # meta_train = [xgb_preds]#ctb_preds, , xgb_preds
        # meta_test = [xgb_test_preds]#ctb_test_preds, , xgb_test_preds
        
        optweights = OptunaWeights(random_state=42, n_trials=300)
        y_val_pred = optweights.fit_predict(y_valid, meta_train)
        print(f"Weight is {optweights.weights}") 
        
        print(f"|------------------------------------|")
        print(f"|                                    |")
        print(f"|            LR Stacking             |")
        print(f"|                                    |")
        print(f"|------------------------------------|")
        print(" ")
        print(f"------------- Train -----------------")
        print(f"BLL: {balanced_log_loss2(y_valid,y_val_pred)}")
        stacking_loss += balanced_log_loss2(y_valid[:],y_val_pred)
        
        
        
        
        
        final_result.append(optweights.predict(meta_test))
        train_result[valid_idx] = y_val_pred
        # weights.append(2/(lgb_loss+xgb_loss))
        weights.append(1/(balanced_log_loss2(y_valid,y_val_pred)))
        score.append(balanced_log_loss2(y_valid,y_val_pred))
    print(f'feature is {X_valid.columns.values}')
    print(f"Average xgb is : {stacking_loss/kfold}")
    print(f"std is {np.std(score)}")
    print(f"total is {np.std(score) + stacking_loss/kfold}")
    return train_result,final_result,weights

model = LogisticRegression(max_iter = 100000)
stacking_train, stacking_test,weight = cv_stacking(train_stack, test_stack.drop(columns = ['tmp']), 'Class',model, kfold = 10)

------------> Fold 1 <-----------------
Weight is [0.00012170968866519099, 0.8644227385772936, 6.2293928929172e-05]
|------------------------------------|
|                                    |
|            LR Stacking             |
|                                    |
|------------------------------------|
 
------------- Train -----------------
BLL: 0.18650094894365035
------------> Fold 2 <-----------------
Weight is [0.0004350129208462745, 0.48981335653510877, 0.0002132983637628531]
|------------------------------------|
|                                    |
|            LR Stacking             |
|                                    |
|------------------------------------|
 
------------- Train -----------------
BLL: 0.07629290703712957
------------> Fold 3 <-----------------
Weight is [2.633656022615134e-06, 0.2649762615528487, 0.6170103751146445]
|------------------------------------|
|                                    |
|            LR Stacking             |
|              

In [13]:
test_preds = np.zeros((test_size,2))
for i in range(10):
    test_preds[:, 1] += weight[i] * stacking_test[i]
test_preds /= sum(weight)
test_preds[:,0] = 1-test_preds[:,1]

In [14]:
submission = pd.concat([test.Id,pd.DataFrame(test_preds,columns = ['class_0','class_1'])],axis=1)
submission.to_csv(r"submission.csv", index=False)